## Find Which Batch Size KO's GPU

In [1]:
from dask.distributed import Client

In [2]:
import os
os.chdir('/home/ubuntu/adadamp-experiments')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
from adadamp.adadamp import DaskClassifier, DaskClassifierIncreasingLR

In [3]:
# training client
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/6337/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [4]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [5]:
# load data - https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Batch Size
GPU has 15,109 MiB total memory. Stats pulled from `nvidia-smi`. Restarted kernel before each run. Diff calculate as: `(curr mem - prev mem)/num_items_added`.
- 128 bs: \~9689MiB
- 256 bs: \~11019MiB (\~ +10MiB/item)
- 512 bs: \~12271MiB (\~ +4MiB/item)
- 640 bs: \~13831MiB (\~ +4Mib/item)
- 1024 bs: \~14749MiB (\~ +2.4MiB/item)
- 2048 bs: crassh
- Conclusion: usee 1024 max BS on this machine

Note: I found that this number starts smaller and grows until it reaches a memory value near the value stated above. It then (lightly) fluctuates near that value.

In [6]:
bs = 2048

In [7]:
device = "cpu" if not torch.cuda.is_available() else "cuda:0"
model = DaskClassifierIncreasingLR(
    module=Wide_ResNet,
    module__depth=16,
    module__widen_factor=4,
    module__dropout_rate=0.3,
    module__num_classes=len(classes),
    loss=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    optimizer__lr=0.1,
    optimizer__momentum=0.9,
    optimizer__nesterov=True,
    optimizer__weight_decay=0.5e-3,
    batch_size=bs,
    max_epochs=200,
    device=device,
    grads_per_worker=bs,
    client=client
)

In [8]:
args = (model, train_set, test_set)
model.fit(train_set)

| Wide-Resnet 16x4
Initial Learning Rate: 0.1


distributed.worker - WARNING -  Compute Failed
Function:  gradient
args:      ((Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer1): Sequential(
    (0): wide_basic(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout): Dropout(p=0.3, inplace=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): wide_basic(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout): Dropout(p=0.3, inplace=False)
      (bn2): BatchNorm2d(64, eps=1e-05, mo

RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 14.76 GiB total capacity; 12.74 GiB already allocated; 401.75 MiB free; 13.33 GiB reserved in total by PyTorch)

In [ ]:
toCSV = model.curr_metas
with open('./exp-dask/exp1-decreaseingLR-const-workers-v0.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
    fc.writeheader()
    fc.writerows(toCSV)